In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten

In [4]:
# Function to load images from directory and create sequences
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize image to a common size
            images.append(img)
    return images

In [5]:
dogs = '/content/drive/MyDrive/images/dog'
cats = '/content/drive/MyDrive/images/cat'

In [6]:
dogs_images = load_images_from_folder(dogs)
cats_images = load_images_from_folder(cats)

In [7]:
# Create labels (1 for dogs, 0 for cats)
dogs_labels = np.ones(len(dogs_images))
cats_labels = np.zeros(len(cats_images))

In [8]:
# Concatenate images and labels
images = np.array(dogs_images + cats_images)
labels = np.concatenate((dogs_labels, cats_labels), axis=0)

In [9]:
# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

In [10]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(219, 64, 64, 3) (219,)
(55, 64, 64, 3) (55,)


In [11]:
# Normalize pixel values
train_images = train_images / 255.0
test_images = test_images / 255.0

In [12]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(219, 64, 64, 3) (219,)
(55, 64, 64, 3) (55,)


In [13]:
# Reshape images into sequences (considering each row as a time step)
time_steps = 64  # Number of time steps (rows)
input_dim = 64    # Dimension of each time step (columns)
input_channels = 3

In [14]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(219, 64, 64, 3) (219,)
(55, 64, 64, 3) (55,)


In [15]:
# Reshape images into sequences for LSTM
train_images = train_images.reshape((-1, time_steps, input_dim * input_channels))
test_images = test_images.reshape((-1, time_steps, input_dim * input_channels))


In [16]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(219, 64, 192) (219,)
(55, 64, 192) (55,)


In [17]:
# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(time_steps, input_dim * input_channels)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)


(219, 64, 192)
(219,)
(55, 64, 192)
(55,)


In [20]:
# Train the model
model.fit(train_images, train_labels, epochs=100, batch_size=32, validation_data=(test_images, test_labels))

Epoch 1/100
7/7 [==============================] - 4s 101ms/step - loss: 0.7029 - accuracy: 0.4840 - val_loss: 0.7024 - val_accuracy: 0.5273
Epoch 2/100
7/7 [==============================] - 0s 18ms/step - loss: 0.6820 - accuracy: 0.5571 - val_loss: 0.6997 - val_accuracy: 0.4909
Epoch 3/100
7/7 [==============================] - 0s 16ms/step - loss: 0.6758 - accuracy: 0.5753 - val_loss: 0.7086 - val_accuracy: 0.4909
Epoch 4/100
7/7 [==============================] - 0s 16ms/step - loss: 0.6618 - accuracy: 0.5616 - val_loss: 0.7150 - val_accuracy: 0.4727
Epoch 5/100
7/7 [==============================] - 0s 16ms/step - loss: 0.6437 - accuracy: 0.6712 - val_loss: 0.7402 - val_accuracy: 0.4182
Epoch 6/100
7/7 [==============================] - 0s 18ms/step - loss: 0.6308 - accuracy: 0.6164 - val_loss: 0.7477 - val_accuracy: 0.4727
Epoch 7/100
7/7 [==============================] - 0s 17ms/step - loss: 0.6173 - accuracy: 0.6804 - val_loss: 0.7662 - val_accuracy: 0.4727
Epoch 8/100
7/7 [==

In [21]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")

2/2 [==============================] - 0s 13ms/step - loss: 3.7905 - accuracy: 0.4545
Test accuracy: 0.4545454680919647


In [25]:
from tensorflow.keras.models import save_model, load_model
import pickle

In [23]:
model.save('dogvscat.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
loaded_model = load_model('dogvscat.h5')

In [27]:
with open('dogandcat.pkl', 'wb') as file:
    pickle.dump(loaded_model, file)